<a href="https://colab.research.google.com/github/taraponglab/deepra/blob/main/DeepRA_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Tutorial for using DeepRA framework in mutagenicity prediction**
## DeepRA: A Novel Deep Learning-Read-Across Framework and Its Application in Non-Sugar Sweeteners Mutagenicity Prediction
Tarapong Srisongkram
* Division of Pharmaceutical Chemistry, Faculty of Pharmaceutical Sciences, Khon Kaen University, 40002 (tarasri@kku.ac.th)

To use this model, please use our sweeteners csv template and change our data to your data, **without changing the column names.**
* **This software requires only chemical name and canonical SMILES**.


In [1]:
1#Download necessary packages
!pip install rdkit
!pip install mordred

In [ ]:
!git clone https://github.com/taraponglab/deepra.git

Cloning into 'deepra'...
remote: Enumerating objects: 21, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 21 (delta 3), reused 11 (delta 2), pack-reused 9
Receiving objects: 100% (21/21), 102.04 MiB | 19.05 MiB/s, done.
Resolving deltas: 100% (4/4), done.


In [12]:
#%cd deepra
!git fetch
!git pull

remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Total 3 (delta 2), reused 3 (delta 2), pack-reused 0
Unpacking objects: 100% (3/3), 269 bytes | 33.00 KiB/s, done.
From https://github.com/taraponglab/deepra
   303e2c4..bd4a5a2  main       -> origin/main
Updating 303e2c4..bd4a5a2
Fast-forward
 mordreds.txt | 2 --
 1 file changed, 2 deletions(-)


In [13]:
%ls

AD_nn.joblib        DeepRA-ECFP.joblib            DeepRA_tutorial.ipynb  train.csv
CNN-ECFP.joblib     DeepRA-Mordred.joblib         dulcin.csv
CNN-Mordred.joblib  DeepRA-Mordred-scaler.joblib  mordred_scaler.joblib
CNN-RDKIT.joblib    DeepRA-RDKIT.joblib           mordreds.txt


In [14]:
#Download models
from joblib import load
scaler_mord = load('mordred_scaler.joblib')
model_mord  = load('CNN-Mordred.joblib')
model_rdkit = load('CNN-RDKIT.joblib')
model_ecfp  = load('CNN-ECFP.joblib')
model_ad    = load('AD_nn.joblib')
deepra      = load('DeepRA-Mordred.joblib')

In [15]:
#Load test data
import pandas as pd
df = pd.read_csv('dulcin.csv', index_col = 'Name')
df

,canonical_smiles
Name,
dulcin,CCOc1ccc(NC(N)=O)cc1


In [16]:
#Calculate Mordred, ECFP, RDKIT
from mordred import Calculator, descriptors
from rdkit import Chem
from rdkit.Chem import AllChem as Chem

def morded_cal(df, smiles_col):
    #calculate descriptors
    calc = Calculator(descriptors, ignore_3D=True)
    mols = [Chem.MolFromSmiles(smi) for smi in df[smiles_col]]
    des = calc.pandas(mols)
    des = des.set_index(df.index)
    with open('mordreds.txt', 'r') as file:
      columns = [line.strip() for line in file]
    selected_mord = des[columns]
    selected_mord
    return selected_mord
def calculate_ecfp(df, smiles_col, radius=10, nBits=4096):
    def get_ecfp(smiles):
        try:
            mol = Chem.MolFromSmiles(smiles)
            fingerprint = Chem.GetMorganFingerprintAsBitVect(mol, radius, nBits)
            return [int(bit) for bit in fingerprint.ToBitString()]
        except:
            return [None] * nBits  # Return a list of None if an error occurs

    ecfp_bits_df = df[smiles_col].apply(get_ecfp).apply(pd.Series)
    ecfp_bits_df.columns = [f'ECFP{i}' for i in range(nBits)]
    df = pd.concat([df, ecfp_bits_df], axis=1)

    return df
def calculate_rdkit(df, smiles_col, nBits=2048):
    def get_ecfp(smiles):
        try:
            mol = Chem.MolFromSmiles(smiles)
            fingerprint = Chem.RDKFingerprint(mol)
            return [int(bit) for bit in fingerprint.ToBitString()]
        except:
            return [None] * nBits  # Return a list of None if an error occurs

    rdkit_bits_df = df[smiles_col].apply(get_ecfp).apply(pd.Series)
    rdkit_bits_df.columns = [f'RDKit{i}' for i in range(nBits)]
    df = pd.concat([df, rdkit_bits_df], axis=1)

    return df

df_mord = morded_cal(df, 'canonical_smiles')
df_ecfp = calculate_ecfp(df, 'canonical_smiles')
df_rdkit= calculate_rdkit(df, 'canonical_smiles')

df_ecfp = df_ecfp.drop(['canonical_smiles'],axis=1)
df_rdkit = df_rdkit.drop(['canonical_smiles'],axis=1)

100%|██████████| 1/1 [00:00<00:00,  6.01it/s]


In [17]:
df_mord.columns[df_mord.isna().any()].tolist()

[]

In [18]:
df_mord.select_dtypes(include=['object']).columns.tolist()

[]

In [19]:
#Compute baseline CNN
import numpy as np
#Mord
name = 'CNN-Mordred'
scaler_mord = load('mordred_scaler.joblib')
x_mord_np = scaler_mord.transform(df_mord)
y_predict_cnn_mord = model_mord.predict(x_mord_np)
y_predict_cnn_mord = np.where(y_predict_cnn_mord > 0.5 ,1 ,0)
y_predict_cnn_mord = pd.DataFrame(y_predict_cnn_mord, columns=[name]).set_index(df_mord.index)
print(y_predict_cnn_mord)
#ECFP
name = 'CNN-ECFP'
x_ecfp_np = np.array(df_ecfp)
y_predict_cnn_ecfp = model_ecfp.predict(x_ecfp_np)
y_predict_cnn_ecfp = np.where(y_predict_cnn_ecfp > 0.5 ,1 ,0)
y_predict_cnn_ecfp = pd.DataFrame(y_predict_cnn_ecfp, columns=[name]).set_index(df_ecfp.index)
print(y_predict_cnn_ecfp)
#RDKIT
name = 'CNN-RDKIT'
x_rdkit_np  = np.array(df_rdkit)
y_predict_cnn_rdkit = model_rdkit.predict(x_rdkit_np)
y_predict_cnn_rdkit = np.where(y_predict_cnn_rdkit > 0.5 ,1 ,0)
y_predict_cnn_rdkit = pd.DataFrame(y_predict_cnn_rdkit, columns=[name]).set_index(df_rdkit.index)
print(y_predict_cnn_rdkit)

1/1 [==============================] - 0s 291ms/step
        CNN-Mordred
Name               
dulcin            1
1/1 [==============================] - 0s 148ms/step
        CNN-ECFP
Name            
dulcin         1
1/1 [==============================] - 0s 140ms/step
        CNN-RDKIT
Name             
dulcin          1


In [20]:
#Compute baseline RA
from rdkit import Chem, DataStructs
from rdkit.Chem import AllChem
train = pd.read_csv('train.csv', index_col='PubChem CID')
y_train = train['Toxicity Value']
def ra_ecfp(train,test, y_train, n_top, smiles_col,weight_power, nBits=4096):
    y_pred = []
    #smiles to morgan fingerprint
    train_smile= list(train[smiles_col])
    test_smile = list(test[smiles_col])
    for compound_test in test_smile:
        mol1 = Chem.MolFromSmiles(compound_test)
        fp1 = AllChem.GetMorganFingerprintAsBitVect(mol1, 8, nBits=4096)
        similarity_array = []
        for compound_train in train_smile:
            mol2 = Chem.MolFromSmiles(compound_train)
            fp2 = AllChem.GetMorganFingerprintAsBitVect(mol2, 8, nBits=4096)
            similarity = DataStructs.TanimotoSimilarity(fp1, fp2)
            similarity_array.append(similarity)
        similarity_series = pd.Series(similarity_array, index = train.index)
        adjusted_weights = similarity_series ** weight_power
        indices = similarity_series.nlargest(n_top).index
        y_values = y_train.loc[indices].values.flatten()  # make sure y_values is 1D
        print(y_values)
        y_pred.append(np.average(y_values, weights=adjusted_weights.loc[indices].values))
    y_pred = pd.DataFrame(y_pred, index=test.index, columns=['ra_ecfp'])
    #set threshold to 0.5
    y_pred['ra_ecfp'] = y_pred['ra_ecfp'].apply(lambda x: 1 if x >= 0.5 else 0)
    return y_pred

y_predict_ra_ecfp = ra_ecfp(train, df, y_train, 3, 'canonical_smiles', 1, nBits=4098)
print(y_predict_ra_ecfp)


def ra_rdkit(train,test, y_train, n_top, smiles_col,weight_power):
    from rdkit import Chem, DataStructs
    from rdkit.Chem import AllChem
    import pandas as pd
    import numpy as np
    y_pred = []
    #smiles to morgan fingerprint
    train_smile= list(train[smiles_col])
    test_smile = list(test[smiles_col])
    for compound_test in test_smile:
        mol1 = Chem.MolFromSmiles(compound_test)
        fp1 = Chem.RDKFingerprint(mol1)
        similarity_array = []
        for compound_train in train_smile:
            mol2 = Chem.MolFromSmiles(compound_train)
            fp2 = Chem.RDKFingerprint(mol2)
            similarity = DataStructs.TanimotoSimilarity(fp1, fp2)
            similarity_array.append(similarity)
        similarity_series = pd.Series(similarity_array, index = train.index)
        adjusted_weights = similarity_series ** weight_power
        indices = similarity_series.nlargest(n_top).index
        y_values = y_train.loc[indices].values.flatten()  # make sure y_values is 1D
        print(y_values)
        y_pred.append(np.average(y_values, weights=adjusted_weights.loc[indices].values))
    y_pred = pd.DataFrame(y_pred, index=test.index, columns=['ra_rdkit'])
    #set threshold to 0.5
    y_pred['ra_rdkit'] = y_pred['ra_rdkit'].apply(lambda x: 1 if x >= 0.5 else 0)
    return y_pred
y_predict_ra_rdkit = ra_rdkit(train, df, y_train, 3, 'canonical_smiles',1)
print(y_predict_ra_rdkit)

[1 1 1]
        ra_ecfp
Name           
dulcin        1
[1 1 1]
        ra_rdkit
Name            
dulcin         1


In [23]:
#Compute DeepRA
name = 'DeepRA-Mordred'
#Mordred + CNN-Mord + CNN-ECFP + CNN-RDKIT + RA-ECFP + RA-RDKIT
x_te   = pd.concat([df_mord, y_predict_ra_ecfp, y_predict_ra_rdkit, y_predict_cnn_ecfp, y_predict_cnn_mord, y_predict_cnn_rdkit], axis=1)
# Transform the test data using the scaler
from sklearn.preprocessing import MinMaxScaler
from joblib import load
scaler_deepra = load('DeepRA-Mordred-scaler.joblib')
x_deepra  = scaler_deepra.transform(x_te)
y_predict_deepra = deepra.predict(x_deepra)

y_predict_deepra_class = np.where(y_predict_deepra > 0.5 ,1 ,0)
y_predict_deepra = pd.DataFrame(y_predict_deepra, columns=[name]).set_index(df.index)
y_predict_deepra_class = pd.DataFrame(y_predict_deepra_class, columns=['class']).set_index(df.index)
y_predict_deepra_class = pd.concat([y_predict_deepra, y_predict_deepra_class], axis=1)
print(y_predict_deepra_class)

1/1 [==============================] - 0s 24ms/step
        DeepRA-Mordred  class
Name                         
dulcin        0.997393      1


In [24]:
#Compute Applicability Domain
def nearest_neighbor_AD_predict(x_te, x_test, k, z=0.5):
    from joblib import load
    import numpy as np
    import pandas as pd
    nn = load('AD_nn.joblib')
    distance, index = nn.kneighbors(x_test)
    #calculate mean and sd of distance in train set
    di = np.mean(distance, axis = 1)
    dk =  1.004
    sk =  0.489
    print('dk = ', dk)
    print('sk = ', sk)
    AD_status = []
    for i in range(len(di)):
        if di[i] < dk + (z * sk):
            AD_status.append('within_AD')
        else:
            AD_status.append('outside_AD')

    # Create DataFrame with index from x_test and the respective status
    df = pd.DataFrame(AD_status, index=x_te.index, columns=['AD_status'])
    return df
test_ad = nearest_neighbor_AD_predict(x_te, x_deepra, 3, z=0.5)
test_ad

dk =  1.004
sk =  0.489


,AD_status
Name,
dulcin,within_AD


In [25]:
#Output
DeepRA_AD = pd.concat([y_predict_deepra_class, test_ad],axis=1)
DeepRA_AD

,DeepRA-Mordred,class,AD_status
Name,,,
dulcin,0.997393,1,within_AD
